In [75]:
#Imports
import pandas as pd
import altair as alt
import numpy as np
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn import metrics
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set_theme(style="ticks")
import matplotlib.pyplot as plt

In [143]:
data = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank.csv')

testClean = data
testClean["job"] = testClean["job"].replace(['unknown'], "admin.")
testClean["marital"] = testClean["marital"].replace(['unknown'], "married")
testClean["education"] = testClean["education"].replace(['unknown'], "university.degree")
testClean["default"] = testClean["default"].replace(['unknown'], "no")
testClean["housing"] = testClean["housing"].replace(['unknown'], "yes")
testClean["loan"] = testClean["loan"].replace(['unknown'], "no")
testClean['poutcome'] = testClean['poutcome'].replace(['nonexistent'], "failure")
testClean['pdays'] = testClean['pdays'].apply(lambda x: np.nan if x == 999 else x)
testClean[['job','marital',"education",'default','housing','contact','month','day_of_week','poutcome','loan','y']] = testClean[['job','marital',"education",'default','housing','contact','month','day_of_week','poutcome','loan','y']].apply(lambda x: pd.factorize(x)[0])
testClean = testClean[~testClean['pdays'].isna()]
testClean.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1373 entries, 21712 to 37060
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             1373 non-null   int64  
 1   job             1373 non-null   int64  
 2   marital         1373 non-null   int64  
 3   education       1373 non-null   int64  
 4   default         1373 non-null   int64  
 5   housing         1373 non-null   int64  
 6   loan            1373 non-null   int64  
 7   contact         1373 non-null   int64  
 8   month           1373 non-null   int64  
 9   day_of_week     1373 non-null   int64  
 10  campaign        1373 non-null   int64  
 11  pdays           1373 non-null   float64
 12  previous        1373 non-null   int64  
 13  poutcome        1373 non-null   int64  
 14  emp.var.rate    1373 non-null   float64
 15  cons.price.idx  1373 non-null   float64
 16  cons.conf.idx   1373 non-null   float64
 17  euribor3m       1373 non-nul

In [142]:
data["y"].value_counts()
#sns.pairplot(testClean, hue="y")
#testClean.isnull().sum()

no     32861
yes     4208
Name: y, dtype: int64

In [135]:
#Splitting the data
X = testClean.iloc[:,14:17]
y = testClean['y']

#Splitting the data into training, validation, and testing
X_main, X_test, y_main, y_test = train_test_split(X, y, test_size=0.15 , random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_main, y_main, test_size=0.15, random_state=42)

In [136]:
#see correlation with validation set and training set
mutual_info = mutual_info_classif(X_train, y_train)
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)
# based on the mutual info, we can see that the most important features are euribor3m,cons.price.idx, cons.conf.idx, and pdays.

cons.price.idx    0.071841
emp.var.rate      0.038484
cons.conf.idx     0.037493
dtype: float64

In [137]:
#selecting the best features
sel_three_cols = SelectKBest(mutual_info_classif, k=3)
sel_three_cols.fit(X_train, y_train)
X_train.columns[sel_three_cols.get_support()]



#Decision Tree
#Decision Tree with max depth of 3
dt = DecisionTreeClassifier(max_depth=15)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_val)
print("Accuracy:",metrics.accuracy_score(y_val, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))
print("Classification Report:")
print(metrics.classification_report(y_val, y_pred))

# 0 is no and 1 is yes

Accuracy: 0.6818181818181818
Confusion Matrix:
[[ 18  51]
 [  5 102]]
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.26      0.39        69
           1       0.67      0.95      0.78       107

    accuracy                           0.68       176
   macro avg       0.72      0.61      0.59       176
weighted avg       0.71      0.68      0.63       176

